### Query Cleansing

In [17]:
import functions as fn

from pymongo import MongoClient
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import numpy as np

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster




In [25]:
#setting mongo
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("lab")

In [27]:
#extract method
def extract_company(filter_,c):
    projection = {"_id":0, "name":1, "category_code":1, "tag_list":1, "total_money_raised":1, "offices.city":1, "offices.state_code": 1, "offices.country_code":1,"offices.latitude": 1, "offices.longitude": 1}
    list_ = list(c.find(filter_, projection))
    
    df = pd.DataFrame(list_).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df = df.drop(["offices"], axis = 1)
    df = df[df["city"]!=""]
    
    return df



In [28]:
#Mongo query for 1M +
million_filter = {"total_money_raised" : {"$regex" : "[$€].*[MB]"}}
money_df = extract_company(million_filter, c)
money_df["city"].value_counts()[:5]

San Francisco    436
New York         335
London           173
Seattle          110
Mountain View    106
Name: city, dtype: int64

In [31]:
#mongo query for desing
design_filter = {"tag_list" : {"$regex" : "design"}}
design_df = extract_company(design_filter,c)
design_df["city"].value_counts()[:5]

London           24
New York         23
San Francisco    20
San Diego        11
Chennai          10
Name: city, dtype: int64

In [32]:
#mongo query for game companies
gaming_hub_filter = {"category_code" : "games_video"}
gaming_df = extract_company(gaming_hub_filter,c)
gaming_df["city"].value_counts()[:5]

New York         75
San Francisco    68
London           36
Los Angeles      31
Paris            17
Name: city, dtype: int64

### LONDON

In [34]:
#quering mongo for London company and making a dataframe
london_filter = {"offices.0.city" : "London"}
london = extract_company(london_filter,c)
london = london[london["city"]=="London"]
london.dropna(subset=["latitude"], inplace=True)
london.dropna(subset=["longitude"], inplace=True)
london.reset_index(drop=True, inplace=True)
london

,name,category_code,tag_list,total_money_raised,city,state_code,country_code,latitude,longitude
0,Babelgum,games_video,"iptv, web2ireland",$13.2M,London,None,GBR,53.344104,-6.267494
1,Curverider,network_hosting,"socialnetwork, opensource",£300k,London,None,GBR,51.785428,-1.197534
2,Zopa,finance,prosper,$33.9M,London,None,GBR,51.517904,-0.139947
3,Kelkoo,ecommerce,"ecommerce, pricecomparison, yahoo",$3M,London,None,USA,37.090240,-95.712891
4,ReachLocal,advertising,"internet-marketing, online-advertising, web-pr...",$68.1M,London,None,GBR,41.531550,-72.188584
...,...,...,...,...,...,...,...,...,...
243,ArcelorMittal,None,None,$0,London,None,GBR,51.509644,-0.146588
244,Testplant,software,,$2.56M,London,None,GBR,51.517356,-0.103774
245,iomart Group,network_hosting,"cloud-computing, cloud-hosting, data-centres, ...",$0,London,None,GBR,51.523114,-0.084670
246,Block Shield,hardware,,$0,London,None,GBR,51.512790,-0.114374


In [35]:
#setting token
load_dotenv()
token_fsq = os.getenv("token")

In [38]:
def foursquare_cat (category, name, df): # v2
    response_list = []
    distance = []
    lat = []
    lon = []
    
    for i in range(len(df)):
        url = f"https://api.foursquare.com/v3/places/search?ll={df['latitude'][i]}%2C{df['longitude'][i]}&categories={category}&limit=1"
        headers = {"accept": "application/json", "Authorization": token_fsq}
        response = requests.get(url, headers=headers).json()
        
        response_list.append(response)
        
    for x in range(len(response_list)):
        try:
            distance.append(response_list[x]["results"][0]["distance"])
        except:
            distance.append(None)
            

    for x in range(len(response_list)):
        try:
            lat.append(response_list[x]["results"][0]["geocodes"]["main"]["latitude"])
        except:
            lat.append(None)
            

    for x in range(len(response_list)):
        try:
            lon.append(response_list[x]["results"][0]["geocodes"]["main"]["longitude"])


        except:
            lon.append(None)


            

        
    df[f"{name}_dist"] = distance
    df[f"{name}_lat"] = lat
    df[f"{name}_lon"] = lon


    
    return df

In [39]:
def foursquare_query (query, df): # v2
    response_list = []
    distance = []
    lat = []
    lon = []
    
    for i in range(len(df)):
        url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={df['latitude'][i]}%2C{df['longitude'][i]}&sort=DISTANCE&limit=1"
        headers = {"accept": "application/json", "Authorization": token_fsq}
        response = requests.get(url, headers=headers).json()
        
        response_list.append(response)

        
        
    for x in range(len(response_list)):
        try:
            distance.append(response_list[x]["results"][0]["distance"])
        except:
            distance.append(None)
            

    for x in range(len(response_list)):
        try:
            lat.append(response_list[x]["results"][0]["geocodes"]["main"]["latitude"])
        except:
            lat.append(None)
            

    for x in range(len(response_list)):
        try:
            lon.append(response_list[x]["results"][0]["geocodes"]["main"]["longitude"])


        except:
            lon.append(None)
        
    df[f"{query}_dist"] = distance
    df[f"{query}_lat"] = lat
    df[f"{query}_lon"] = lon
    
    
    return df

In [37]:
category = 19040 #airport
name = "airport"
nearest_park = foursquare_cat(category, name, london)

NameError: name 'foursquare_cat' is not defined

In [64]:
query = "starbucks" #Starbucks
nearest_starbucks = foursquare_query(query, london)

In [65]:
category = 10032 #nightclub
name = "n_club"
nearest_club = foursquare_cat(category, name, london)

In [67]:
query = 18007 #Basketball Court
name = "b_stadium"
nearest_b_stadium = foursquare_cat(category, name, london)

In [68]:
category = 18006 #elementary school
name = "school"
nearest_b_stadium = foursquare_cat(category, name, london)

In [73]:
final_df2 = nearest_b_stadium 

In [74]:
final_df2

,name,category_code,tag_list,total_money_raised,city,state_code,country_code,latitude,longitude,airport_dist,...,starbucks_lon,n_club_dist,n_club_lat,n_club_lon,b_stadium_dist,b_stadium_lat,b_stadium_lon,school_dist,school_lat,school_lon
0,Babelgum,games_video,"iptv, web2ireland",$13.2M,London,None,GBR,53.344104,-6.267494,NaN,...,-6.263846,328.0,53.345005,-6.262935,328.0,53.345005,-6.262935,2657.0,53.367636,-6.275254
1,Curverider,network_hosting,"socialnetwork, opensource",£300k,London,None,GBR,51.785428,-1.197534,NaN,...,-1.222343,5498.0,51.752249,-1.256815,5498.0,51.752249,-1.256815,4301.0,51.746812,-1.203794
2,Zopa,finance,prosper,$33.9M,London,None,GBR,51.517904,-0.139947,22705.0,...,-0.142731,760.0,51.513464,-0.131633,760.0,51.513464,-0.131633,3331.0,51.496187,-0.106748
3,Kelkoo,ecommerce,"ecommerce, pricecomparison, yahoo",$3M,London,None,USA,37.090240,-95.712891,NaN,...,NaN,8235.0,37.036619,-95.649091,8235.0,37.036619,-95.649091,NaN,NaN,NaN
4,ReachLocal,advertising,"internet-marketing, online-advertising, web-pr...",$68.1M,London,None,GBR,41.531550,-72.188584,NaN,...,-72.111818,10685.0,41.553463,-72.314036,10685.0,41.553463,-72.314036,29627.0,41.563276,-72.542324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,ArcelorMittal,None,None,$0,London,None,GBR,51.509644,-0.146588,22059.0,...,-0.144165,34.0,51.509188,-0.146869,34.0,51.509188,-0.146869,3135.0,51.496187,-0.106748
244,Testplant,software,,$2.56M,London,None,GBR,51.517356,-0.103774,25132.0,...,-0.103198,1976.0,51.513464,-0.131633,1976.0,51.513464,-0.131633,2361.0,51.496187,-0.106748
245,iomart Group,network_hosting,"cloud-computing, cloud-hosting, data-centres, ...",$0,London,None,GBR,51.523114,-0.084670,26559.0,...,-0.083700,3422.0,51.513464,-0.131633,3422.0,51.513464,-0.131633,3359.0,51.496187,-0.106748
246,Block Shield,hardware,,$0,London,None,GBR,51.512790,-0.114374,24312.0,...,-0.117479,1196.0,51.513464,-0.131633,1196.0,51.513464,-0.131633,1918.0,51.496187,-0.106748


In [75]:
#droping columns
df2 = final_df2.drop(["category_code","tag_list","total_money_raised","state_code", "country_code"], axis=1)

In [76]:
df2

,name,city,latitude,longitude,airport_dist,airport_lat,airport_lon,starbucks_dist,starbucks_lat,starbucks_lon,n_club_dist,n_club_lat,n_club_lon,b_stadium_dist,b_stadium_lat,b_stadium_lon,school_dist,school_lat,school_lon
0,Babelgum,London,53.344104,-6.267494,NaN,NaN,NaN,354.0,53.341806,-6.263846,328.0,53.345005,-6.262935,328.0,53.345005,-6.262935,2657.0,53.367636,-6.275254
1,Curverider,London,51.785428,-1.197534,NaN,NaN,NaN,2858.0,51.764916,-1.222343,5498.0,51.752249,-1.256815,5498.0,51.752249,-1.256815,4301.0,51.746812,-1.203794
2,Zopa,London,51.517904,-0.139947,22705.0,51.470012,-0.454478,194.0,51.517657,-0.142731,760.0,51.513464,-0.131633,760.0,51.513464,-0.131633,3331.0,51.496187,-0.106748
3,Kelkoo,London,37.090240,-95.712891,NaN,NaN,NaN,NaN,NaN,NaN,8235.0,37.036619,-95.649091,8235.0,37.036619,-95.649091,NaN,NaN,NaN
4,ReachLocal,London,41.531550,-72.188584,NaN,NaN,NaN,6759.0,41.511614,-72.111818,10685.0,41.553463,-72.314036,10685.0,41.553463,-72.314036,29627.0,41.563276,-72.542324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,ArcelorMittal,London,51.509644,-0.146588,22059.0,51.470012,-0.454478,188.0,51.508664,-0.144165,34.0,51.509188,-0.146869,34.0,51.509188,-0.146869,3135.0,51.496187,-0.106748
244,Testplant,London,51.517356,-0.103774,25132.0,51.470012,-0.454478,93.0,51.516398,-0.103198,1976.0,51.513464,-0.131633,1976.0,51.513464,-0.131633,2361.0,51.496187,-0.106748
245,iomart Group,London,51.523114,-0.084670,26559.0,51.470012,-0.454478,356.0,51.519997,-0.083700,3422.0,51.513464,-0.131633,3422.0,51.513464,-0.131633,3359.0,51.496187,-0.106748
246,Block Shield,London,51.512790,-0.114374,24312.0,51.470012,-0.454478,257.0,51.513847,-0.117479,1196.0,51.513464,-0.131633,1196.0,51.513464,-0.131633,1918.0,51.496187,-0.106748


In [87]:
#droping all the rows with nan

df2 = df2.dropna()
df2.reset_index(drop=True, inplace=True)

df2

,name,city,latitude,longitude,airport_dist,airport_lat,airport_lon,starbucks_dist,starbucks_lat,starbucks_lon,n_club_dist,n_club_lat,n_club_lon,b_stadium_dist,b_stadium_lat,b_stadium_lon,school_dist,school_lat,school_lon,score
0,Zopa,London,51.517904,-0.139947,22705.0,51.470012,-0.454478,194.0,51.517657,-0.142731,760.0,51.513464,-0.131633,760.0,51.513464,-0.131633,3331.0,51.496187,-0.106748,0.179242
1,Seedcamp,London,51.510880,-0.141897,22404.0,51.470012,-0.454478,200.0,51.510415,-0.139072,376.0,51.509188,-0.146869,376.0,51.509188,-0.146869,2928.0,51.496187,-0.106748,0.222839
2,Wonga,London,51.519204,-0.162610,21220.0,51.470012,-0.454478,379.0,51.522717,-0.163106,1549.0,51.509188,-0.146869,1549.0,51.509188,-0.146869,4633.0,51.496187,-0.106748,0.092130
3,FAROO,London,51.519579,-0.106555,24997.0,51.470012,-0.454478,220.0,51.520270,-0.103682,1865.0,51.513464,-0.131633,1865.0,51.513464,-0.131633,2599.0,51.496187,-0.106748,0.137451
4,Dopplr,London,51.523789,-0.087432,26390.0,51.470012,-0.454478,267.0,51.526229,-0.088148,3267.0,51.513464,-0.131633,3267.0,51.513464,-0.131633,3345.0,51.496187,-0.106748,0.108472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,ArcelorMittal,London,51.509644,-0.146588,22059.0,51.470012,-0.454478,188.0,51.508664,-0.144165,34.0,51.509188,-0.146869,34.0,51.509188,-0.146869,3135.0,51.496187,-0.106748,1.166940
225,Testplant,London,51.517356,-0.103774,25132.0,51.470012,-0.454478,93.0,51.516398,-0.103198,1976.0,51.513464,-0.131633,1976.0,51.513464,-0.131633,2361.0,51.496187,-0.106748,0.291959
226,iomart Group,London,51.523114,-0.084670,26559.0,51.470012,-0.454478,356.0,51.519997,-0.083700,3422.0,51.513464,-0.131633,3422.0,51.513464,-0.131633,3359.0,51.496187,-0.106748,0.084559
227,Block Shield,London,51.512790,-0.114374,24312.0,51.470012,-0.454478,257.0,51.513847,-0.117479,1196.0,51.513464,-0.131633,1196.0,51.513464,-0.131633,1918.0,51.496187,-0.106748,0.132988


In [78]:
# my valors 100% importats airport school and starbucks 30% 25% 20% 15% 10%
def distance(df):
    score = []
    for i in range(len(df)):
        airport = df["airport_dist"][i]
        starbucks = df["starbucks_dist"][i]
        night_club = df["n_club_dist"][i]
        basket = df["b_stadium_dist"][i]
        school = df["school_dist"][i]
        total_score = (1/airport * 0.30)*100 + (1/starbucks * 0.25)*100 + (1/night_club * 0.20)*100 + (1/basket * 0.15)*100 + (1/school * 0.10)*100
        score.append(total_score)
        
    df["score"] = score
    
    return df
            
            
            

In [79]:
#using a function to make the score column
df3 = distance(df2)

/var/folders/b0/dq4wb7ld55l9pqp5_v7jcgmc0000gn/T/ipykernel_45440/3718538884.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["score"] = score


In [80]:
df3

,name,city,latitude,longitude,airport_dist,airport_lat,airport_lon,starbucks_dist,starbucks_lat,starbucks_lon,n_club_dist,n_club_lat,n_club_lon,b_stadium_dist,b_stadium_lat,b_stadium_lon,school_dist,school_lat,school_lon,score
0,Zopa,London,51.517904,-0.139947,22705.0,51.470012,-0.454478,194.0,51.517657,-0.142731,760.0,51.513464,-0.131633,760.0,51.513464,-0.131633,3331.0,51.496187,-0.106748,0.179242
1,Seedcamp,London,51.510880,-0.141897,22404.0,51.470012,-0.454478,200.0,51.510415,-0.139072,376.0,51.509188,-0.146869,376.0,51.509188,-0.146869,2928.0,51.496187,-0.106748,0.222839
2,Wonga,London,51.519204,-0.162610,21220.0,51.470012,-0.454478,379.0,51.522717,-0.163106,1549.0,51.509188,-0.146869,1549.0,51.509188,-0.146869,4633.0,51.496187,-0.106748,0.092130
3,FAROO,London,51.519579,-0.106555,24997.0,51.470012,-0.454478,220.0,51.520270,-0.103682,1865.0,51.513464,-0.131633,1865.0,51.513464,-0.131633,2599.0,51.496187,-0.106748,0.137451
4,Dopplr,London,51.523789,-0.087432,26390.0,51.470012,-0.454478,267.0,51.526229,-0.088148,3267.0,51.513464,-0.131633,3267.0,51.513464,-0.131633,3345.0,51.496187,-0.106748,0.108472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,ArcelorMittal,London,51.509644,-0.146588,22059.0,51.470012,-0.454478,188.0,51.508664,-0.144165,34.0,51.509188,-0.146869,34.0,51.509188,-0.146869,3135.0,51.496187,-0.106748,1.166940
225,Testplant,London,51.517356,-0.103774,25132.0,51.470012,-0.454478,93.0,51.516398,-0.103198,1976.0,51.513464,-0.131633,1976.0,51.513464,-0.131633,2361.0,51.496187,-0.106748,0.291959
226,iomart Group,London,51.523114,-0.084670,26559.0,51.470012,-0.454478,356.0,51.519997,-0.083700,3422.0,51.513464,-0.131633,3422.0,51.513464,-0.131633,3359.0,51.496187,-0.106748,0.084559
227,Block Shield,London,51.512790,-0.114374,24312.0,51.470012,-0.454478,257.0,51.513847,-0.117479,1196.0,51.513464,-0.131633,1196.0,51.513464,-0.131633,1918.0,51.496187,-0.106748,0.132988


In [81]:
#sorting x score and only using the 5 better scores
df4 = df3.sort_values(by=['score'],ascending=False).head()
df4

,name,city,latitude,longitude,airport_dist,airport_lat,airport_lon,starbucks_dist,starbucks_lat,starbucks_lon,n_club_dist,n_club_lat,n_club_lon,b_stadium_dist,b_stadium_lat,b_stadium_lon,school_dist,school_lat,school_lon,score
99,PCIQ,London,51.515374,-0.130109,23305.0,51.470012,-0.454478,16.0,51.515100,-0.130180,242.0,51.513464,-0.131633,242.0,51.513464,-0.131633,2675.0,51.496187,-0.106748,1.712154
138,Filmed Media,London,51.535901,-0.205508,19001.0,51.470012,-0.454478,15.0,51.535908,-0.205126,5020.0,51.509188,-0.146869,5020.0,51.509188,-0.146869,8131.0,51.496187,-0.106748,1.676448
192,Zensify,London,51.494076,-0.146659,21778.0,51.470012,-0.454478,21.0,51.494088,-0.146976,1697.0,51.509188,-0.146869,1697.0,51.509188,-0.146869,2771.0,51.496187,-0.106748,1.216087
45,ProcServe,London,51.494067,-0.146665,21778.0,51.470012,-0.454478,21.0,51.494088,-0.146976,1698.0,51.509188,-0.146869,1698.0,51.509188,-0.146869,2771.0,51.496187,-0.106748,1.216075
224,ArcelorMittal,London,51.509644,-0.146588,22059.0,51.470012,-0.454478,188.0,51.508664,-0.144165,34.0,51.509188,-0.146869,34.0,51.509188,-0.146869,3135.0,51.496187,-0.106748,1.166940


In [88]:
#exporting csv
df4.to_csv("data/five_locations.csv", index = False, encoding='unicode_escape')